# Generación de mapas utilizando los datos de latlon según idrutaestacion

Este script tiene por trabajo generar mapas ubicando todos los puntos de (lat,lon) cargados en el dataset utlizando idrutaestacion como parametro de agrupación. Esto muestra todos los puntos sin discrimiar para cada ruta.
La expectativa es que esto ayude a visualizar e identificar cada linea según su ruta, pero probablemente deba refinarse significativamente más.

Importamos las librerias que vamos a utlizar, en particular las que son para trabajar con mapas

In [1]:
import pandas as pd
import numpy as np

import os

import geopandas as gpd
import contextily as cx

import matplotlib.pyplot as plt

Ahora comenzamos a cargar los datos de trabajo, estos son todos los puntos únicos donde se tomo una lectura.

In [2]:
# datos_lineas = pd.read_csv("../../data/processed/latlon-por-linea-clean.csv", sep = ';',low_memory=False)
datos_lineas = pd.read_csv("../../data/interim/latlon-por-linea-full.csv", sep = ';')

Ahora comenzamos a realizar limpieza de datos. Quitamos todo dato no valido, convertimos a latlon a flotantes, quitamos todos los latlon que sean cero; todos los latlon que no estén dentro de Paraguay.

In [3]:
datos_lineas.dropna(how='any', inplace=True) # BORRAR NaNs

datos_lineas = datos_lineas[datos_lineas['latitude'] != 'latitude']  # Remover textos de la columna

datos_lineas['latitude']=datos_lineas['latitude'].astype('float')
datos_lineas['longitude']=datos_lineas['longitude'].astype('float')

datos_lineas = datos_lineas[(datos_lineas[['latitude','longitude']] != 0).all(axis=1)] # Remover ceros
datos_lineas = datos_lineas[datos_lineas['latitude'] < -25.05]  # Remover latitudes fuera de Asunción
datos_lineas = datos_lineas[datos_lineas['longitude'] < -57.4] # Remover longitudes fuera de Asunción
datos_lineas = datos_lineas[datos_lineas['latitude'] > -25.8]  # Remover latitudes fuera de Asunción
datos_lineas = datos_lineas[datos_lineas['latitude'] > -58]  # Remover latitudes fuera de Asunción


print(datos_lineas.sample(10))

           latitude  longitude idrutaestacion
201524134 -25.28744  -57.57238           0117
189491759 -25.30590  -57.64613           0000
95806090  -25.27948  -57.64258           0043
164378192 -25.26468  -57.54744           013d
185579378 -25.29892  -57.62527           004d
103691444 -25.30700  -57.58403           0141
161738286 -25.22865  -57.53531           00b7
56150765  -25.39630  -57.54480           0051
186134313 -25.29874  -57.57909           0116
51720872  -25.17379  -57.44218           00da


In [4]:
lineas = datos_lineas['idrutaestacion'].unique()
np.random.shuffle(lineas)
print(lineas.shape)
print(lineas)

(457,)
['00a2' '00bf' '00f7' '00d2' '007d' '00c7' '0061' '005c' '00f5' '00d4'
 '0045' '0056' '0105' '0014' '00bb' '00ff' '015f' '0113' '00b4' '0130'
 '0032' '0189' '012a' '0023' '011b' '0182' '00af' '014a' '00d0' '008d'
 '0100' '0038' '0190' '00c0' '0126' '016d' '00b9' '00a4' '00fd' '00a5'
 '0177' '013f' '00e4' '0042' '010a' '0079' '015d' '0107' '0160' '013e'
 '0162' '0194' '2714' '003c' '0110' '0141' '016e' '0137' '0066' '0109'
 '014c' 'fdeb' '00a0' '007b' '0030' '01b0' '00da' '004b' '0029' '0115'
 '0018' '00e9' '0138' '00e1' '003e' '0116' '0106' '005d' 'fde8' '009c'
 '0124' '0035' '0022' '0104' '0184' '0154' '00a7' '005f' '0002' '00cc'
 'fe02' '0187' '27b1' '2b08' '2713' '005b' '0170' '00ed' '006e' '0077'
 '0140' '0028' 'fdfb' '00fe' 'fe00' '006c' '017e' '0013' 'fdf4' '0012'
 '0046' '01bb' '0179' '0120' '017d' '0001' '001d' '0053' '0185' '01b9'
 '0153' '00b3' '001a' '016c' '00d9' '017a' '0195' '000d' '00f1' 'fe08'
 '009d' '000f' '00e5' '00ce' '03f2' '0005' 'fdfd' '004c' '003d' '011e'

Comenzamos a crear los mapas para cada idrutaestacion, uno a la vez. Este proceso puede llevar cierto trabajo manual de limpieza y depuración para obtener los mejores resultados; o puede remover el alpha y mostrar todos los puntos y no preocuparse por eso.

In [28]:
print("Comenzando a generar gráficos")
for linea in lineas:
    
    imgpath = './graphs/'+linea+'.png'
    
    if(os.path.isfile(imgpath)):
        continue
    
    print(imgpath)
    
    datos_linea = datos_lineas.loc[datos_lineas['idrutaestacion'] == linea]
    latlon_por_linea= gpd.GeoDataFrame(datos_linea, geometry=gpd.points_from_xy(datos_linea.longitude,datos_linea.latitude),crs="EPSG:4326")

    linea_mapa_plot = latlon_por_linea.plot(figsize=(9, 9),legend=True,marker='.',markersize=0.2,color='red',alpha=.1)
    # linea_mapa_plot = latlon_por_linea.plot(figsize=(9, 9),legend=True,marker='X',markersize=4,color='blue') # MODO ALTA VISIBILIDAD
    linea_mapa_plot.set(title='Datos para idrutaestacion '+linea)
    cx.add_basemap(linea_mapa_plot, crs="EPSG:4326")
    # cx.add_basemap(linea_mapa_plot, crs="EPSG:4326",source=cx.providers.CartoDB.Voyager)
    
    plt.savefig(imgpath,dpi=300)
    plt.close()
    # break
    # exit()

print("Gráficos generados exitosamente.")

Comenzando a generar gráficos
Gráficos generados exitosamente.


Se decide crear un segundo set de mapas en color azul ya que muchos recorridos aparentaban mostrar el uso de rutas variadas pero por contar con pocos puntos de lectura no se podían distinguir en el mapa generado. Se puede distinguir entre ambos tipos de mapa ya que los que cuentan con mucha información son rojos y los demás azules. Esta inspección se realiza de forma manual, así como la eliminación de mapas poco visibles para generar las versiones de alta visibilidad en azul.

In [ ]:
A continuación, vamos a generar un readme.md que muestre todos los gráficos generados junto con su información complementaria.

In [95]:
referencias = pd.read_csv("../../data/processed/idrutaestacion.csv", sep = ',')
md = []
for image in os.listdir("./graphs/"):
    if (image.endswith(".png")):
        ref = image[:len(image) - 4]
        datos_ruta=((referencias[referencias['idruta'].str.contains(ref.upper())]).values)
        titulo=ref
        if(len(datos_ruta)>0):
            datos_ruta=datos_ruta[0]
            titulo=(datos_ruta[2]+": "+datos_ruta[1]+"("+datos_ruta[0]+")")

        md.append('### %s \n' % titulo)
        md.append('![%s]' % titulo)
        md.append('(https://raw.githubusercontent.com/pabloacastillo/billetaje-electronico-py-2022/main/notebooks/mapas-por-idrutaestacion/graphs/%s)' % image)
        md.append('\n')
        # md+=('!['+titulo+'](https://raw.githubusercontent.com/pabloacastillo/billetaje-electronico-py-2022/main/notebooks/mapas-por-idrutaestacion/graphs/'+image+' "'+titulo+'")')

with open('./rutas_imagenes.md', 'w', encoding='utf-8') as f:
    f.writelines(md)
    
print('Listo')      

Listo
